In [ ]:
# (Chapter) SSA

In [ ]:
# (Section) Modules

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy import interpolate

In [ ]:
# (Section) Lexicon

# https://docs.python.org/3/library/collections.html
# Catalyzer
# Building Materials
# Lower-Case Letters
# Upper-Case Letters
# Initialize
# Construct
# Assemble
# Build

# Stoichiometry
# Species
# Initial Copy Numbers
# Copy Numbers ~ Upper-Case Letters
# Chemical Species ~ Lower-Case Letters
# Chemical Reactions
# Event
# Propensity Functions
# Times = Epochs
# Reactions
# Samples
# Steps
# Random Matrix
# Rows = i
# Columns = j
# Others = {p, q, r, s, t}
# Rate Constants
# Reaction Rates
# [(Stochastic) Realizations, Sample Functions, Sample Paths, Trajectories, (Stochastic) Simulations]

# BiochemAnalysis
# Reaction
# Reactant
# Product
# Rate
# Fast
# Slow
# Large
# Small
# Trial
# State
# Tensor
# Array
# Formula
# Equation
# Coefficient
# Novel
# Refresh

# https://docs.python.org/3.9/tutorial/classes.html
# Abstract Class (Method)
# Attribute (Reference)
# Argument
# (Class, Instance) ~ (Object, Variable)

In [ ]:
# (Section) Obsolete

In [ ]:
# (Class) Base

class Base:
    
    def __init__(self, steps, realizations, species, reactions):
        self.steps = steps
        self.realizations = realizations
        _temp = np.full(shape = (steps, 2 + len(species)), fill_value = np.nan)
        temp = np.append(arr = np.resize(a = [0, np.nan] + list(species.values()), new_shape = (1, 2 + len(species))), values = _temp, axis = 0)
        self.conus = pd.DataFrame(data = temp, columns = ['Epochs', 'Reactions'] + list(species))
        self.species = species
        self.reactions = reactions
        self.coni = [np.nan] * realizations
    
    def _ran_numb_gen(self):
        temp = np.random.uniform(size = (self.steps, 2))
        self.ran_mat = pd.DataFrame(data =  temp, columns = ['_Epochs', '_Reactions'])
        return self
    
    def _prop_funs(self, i):
        A = self.conus['A'][i]
        rates = self.reactions
        alps = {'alp_1': A * rates['Degradation'], 'alp_2': rates['Production']}
        alp_0 = sum(list(alps.values()))
        self.alps = alps
        self.alp_0 = alp_0
        return self
    
    def _next_epoch(self, i):
        tau = np.log(1 / self.ran_mat['_Epochs'][i]) / self.alp_0
        self.conus['Epochs'][i + 1] = self.conus['Epochs'][i] + tau
        return self
    
    def _next_reaction(self, i):
        _temp = np.cumsum(list(self.alps.values())) / self.alp_0
        temp = _temp.tolist()
        self.disc = self.ran_mat['_Reactions'][i] < temp[0]
        return self
    
    def _update_conus(self, i):
        op = -1 if self.disc else 1
        self.conus['A'][i + 1] = self.conus['A'][i] + op
        return self
    
    def _update_coni(self, r):
        self.coni[r] = self.conus.copy()
        return self
    
    def meth_direct(self):
        for r in range(self.realizations):
            self._ran_numb_gen()
            for i in range(self.steps):
                self._prop_funs(i)
                self._next_epoch(i)
                self._next_reaction(i)
                self._update_conus(i)
            self._update_coni(r)
        return self
    
    def _collect_epochs(self):
        _epochs = []
        for r in range(self.realizations):
            _epochs.append(self.coni[r][['Epochs']])
        self._epochs = _epochs
        return self
    
    def _collect_species(self):
        _species = []
        for r in range(self.realizations):
            _species.append(self.coni[r][list(self.species)])
        self._species = _species
        return self
    
    def show(self):
        self._collect_epochs()
        self._collect_species()
        for r in range(self.realizations):
            plt.plot(self._epochs[r], self._species[r])
        return 'Plot!'
    
    def test(self):
        self._ran_numb_gen()
        return self.ran_mat.head()
    

In [ ]:
# Example # (Class) Base

species = {'A': 0} # species = {'A': 0, 'B': 0}
reactions = {'Degradation': 0.1, 'Production': 10}

In [ ]:
%%time
base = Base(2500, 10, species, reactions)
base.meth_direct()

In [ ]:
base.show()

In [ ]:
# (Class) Model

class Model(Base):
    
    def _prop_funs(self, i):
        A = self.conus['A'][i]
        B = self.conus['B'][i]
        rates = self.reactions
        alps = {'alp_1': A*(A-1)*rates['A+A-k1->0'], 'alp_2': A*B*rates['A+B-k2->0'], 'alp_3': rates['0-k3->A'], 'alp_4': rates['0-k4->B']}
        alp_0 = sum(list(alps.values()))
        self.alps = alps
        self.alp_0 = alp_0
        return self
    
    def _next_reaction(self, i):
        _temp = np.cumsum(list(self.alps.values())) / self.alp_0
        temp = np.argwhere(self.ran_mat['_Reactions'][i] < _temp)
        if not any(temp):
            disc = len(_temp)
        else:
            disc = temp[0, 0]
        self.disc = disc
        return self
    
    def _update_conus(self, i):
        ops = {'A': [-2, -1, 1, 0], 'B': [0, -1, 0, 1]}
        self.conus['A'][i + 1] = self.conus['A'][i] + ops['A'][self.disc]
        self.conus['B'][i + 1] = self.conus['B'][i] + ops['B'][self.disc]
        return self
    

In [ ]:
# Example # (Class) Model

species = {'A': 0, 'B': 0}
reactions = {'A+A-k1->0': pow(10, -3), 'A+B-k2->0': pow(10, -2), '0-k3->A': 1.2, '0-k4->B': 1}

In [ ]:
model = Model(500, 5, species, reactions)
model.meth_direct()
model.show()

In [ ]:
# (Class) BiochemStem [Biochemical System]

class BiochemStem:
    
    """Class 'BiochemStem' Illustration!
    It is an essential part of the 'BiochemSimul' class!
    
    ########
    Attributes
        rates: 'dict'
            A dictionary containing every reaction rate for the given bio/chemical system.
            Please, use the following convention: lower-case letter and integer index for each rate constant, e.g. 'k0'.
            Each dictionary (key, value) pair must be like ('str', 'float').
            Example: k = {'k0': 0.1, 'k1': 2}.
    
    ########
    Methods
        add_reaction(description, reactants, products, prop_fun)
            It adds a (new) bio/chemical reaction to the current system.
            Please, read carefully its illustration!
        del_reaction(name)
            It deletes a reaction from the current bio/chemical system based on the 'name' argument.
    
    """
    
    def __init__(self, rates):
        """It constructs our new bio/chemical system.
        The only necessary attribute is 'rates'.
        Also, it instantiates an empty dictionary as a container for the reactions.
        
        """
        self.rates = rates
        self.reactions = {} # Empty dictionary
    
    def add_reaction(self, description, reactants, products, prop_fun):
        """It adds a (new) bio/chemical reaction to the current system.
        Please, read carefully its illustration!
        
        ########
        Arguments
            description: 'dict'
                    dict_keys(['name', 'equation'])
                    dict_values(['str', 'str'])
                It provides a name and an equation (optional) for the new reaction.
                Please, use the following convention: lower-case letter and integer index for the name, e.g. 'r0'.
                Example: d = {'name': 'r1', 'equation': 'Reactants-k0->Products'}.
                Note: the key 'equation' is not compulsory, but it will be used in a future definiton of the 'BiochemStem' class.
            reactants: 'dict'
                It presents the reactant species ['str'] and their initial copy numbers (initial state) ['int', 'float'] involved in the new reaction.
                Convention: upper-case letter for each reactant, integer for each initial copy number.
                Example: r = {'A': 0, 'B' = 10}.
            products: 'dict'
                It presents the product species ['str'] and their copy number changes (when this event occurs) ['int', 'float'] for the new reaction.
                Convention: upper-case letter for each product, integer for each copy number change.
                Example: p = {'C': -1, 'D' = 2}.
            prop_fun: 'str'
                It provides the necessary propensity function for the new reaction.
                It must be of type 'str', and it must represent a valid Python expression.
                Please, be careful and consistent with your definitions!
        
        ########
        Returns
            self.reactions: 'dict'
            It returns a dictionary with an entry (bio/chemical reaction) added to the current system.
        
        """
        
        return self
    
    def del_reaction(self, name):
        """It deletes a reaction from the current bio/chemical system based on the 'name' argument.
        
        ########
        Arguments
            name: 'str'
                We remove the reaction given by 'name' from the current bio/chemical system.
                It must be consistent with the 'description' dictionary provided when the method 'add_reaction' was invoked.
        
        ########
        Returns
            self.reactions: 'dict'
            It returns a dictionary with an entry (bio/chemical reaction) deleted from the current system.
        
        """
        
        return self
    
    def _consistency(self):
        pass
    

In [ ]:
# Example # (Class) BiochemStem

reaction = {}

rates = {'k1': 0.1, 'k2': 10}

description = {'name': 'r1', 'equation': 'A-k1->0'}
reactants = {'A': 100}
products = {'A': -1}
prop_fun = 'A*k1'

description = {'name': 'r2', 'equation': '0-k2->A'}
reactants = {'A': 0}
products = {'A': 1}
prop_fun = 'k2'

In [ ]:
# (Section) Prevalent

In [ ]:
# (Class) BiochemStem

class BiochemStem:
    
    """Class 'BiochemStem' Illustration!
    It is an essential part of the 'BiochemSimul' class!
    
    ########
    Attributes
        initial_state: 'dict'
            A dictionary containing the initial state of the given biochemical system.
            It presents every biochemical species involved in the system, together with its initial copy number.
            Convention: upper-case letter for each reactant/product, non-negative integer for each initial copy number.
            Example: initial_state = {'A': 10, 'Z': 0}.
        rates: 'dict'
            A dictionary containing every reaction rate for the given biochemical system.
            Please, use the following convention: lower-case letter and non-negative integer index for each rate constant, e.g. 'k10'.
            Each dictionary (key, value) pair must be like ('str', 'float').
            Example: rates = {'k1': 0.1, 'k2': 2}.
    
    ########
    Methods
        add_reaction(name, prop_fun, delta, equation = None)
            It adds a (new) biochemical reaction to the current system.
            Please, read carefully its illustration!
        del_reaction(name)
            It deletes a reaction from the current biochemical system based on the 'name' argument.
        assemble(show = True)
            It assembles our biochemical system; it makes it ready for simulation/analysis.
    
    """
    
    def __init__(self, initial_state, rates):
        """It initializes our new biochemical system.
        The only necessary initialization attributes are 'initial_state' and 'rates'.
        Also, it instantiates an empty dictionary as a container for all the reactions.
        The (private) instance variable '_assembled' keeps track of the instance object construction status.
        In other words, whenever a reaction is added to or deleted from the current biochemical system, we will need to construct/assemble it (again) before analyzing it.
        
        """
        self.initial_state = initial_state
        self.rates = rates
        self.reactions = {} # Empty dictionary (obvious)
        self._assembled = False # Flag # Instance variable (private)
    
    def __repr__(self):
        portrait = f'<{self.__class__.__name__}({self.initial_state}, {self.rates})>'
        return portrait
    
    def __str__(self):
        portrait = repr(self)
        return portrait
    
    def _consistency(self, temp, checkers, verbose):
        for key, value in self.initial_state.items():
            express = '{0} = {1}'.format(key, value)
            exec(express)
        for key, value in self.rates.items():
            express = '{0} = {1}'.format(key, value)
            exec(express)
        _consistent = {key: False for key in checkers} # User info # Default
        _err = 'Unexpected error!\nTry again...' # Default error message
        _express = "print('Is <{0}> consistent?\t{1}'.format(checker, _consistent[checker]))"
        for checker in checkers:
            check = temp[checker]
            if checker == 'prop_fun':
                try:
                    _ = eval(check)
                except NameError as err:
                    _message = "'{}' is using some species/rates not yet defined".format(checker)
                    _err = 'Error!\n{0}\n{1}'.format(err, _message)
                    print(_err)
                except SyntaxError as err:
                    _message = "'{}' is not a valid Python expression".format(checker)
                    _err = 'Error!\n{0}\n{1}'.format(err, _message)
                    print(_err)
                except:
                    print(_err)
                else:
                    _consistent[checker] = True
                finally:
                    exec(_express) if verbose else None
            if checker == 'delta':
                try:
                    _ = {self.initial_state[key] for key in check}
                except KeyError as err:
                    _message = "'{0}' is trying to change some unknown species: {1}".format(checker, err)
                    _err = 'Error!\n{}'.format(_message)
                    print(_err)
                except:
                    print(_err)
                else:
                    _consistent[checker] = True
                finally:
                    exec(_express) if verbose else None
        return _consistent
    
    def add_reaction(self, name, prop_fun, delta, equation = None, verbose = False):
        """It adds a (new) biochemical reaction to the current system.
        Please, read carefully its illustration!
        
        ########
        Arguments
            name: 'str'
                It provides a name for the new reaction.
                Please, use the following convention: lower-case letter and non-negative integer index for the name, e.g. 'r10'.
            prop_fun: 'str'
                It provides the necessary propensity function for the new reaction.
                It must be of type 'str', and it must represent a valid Python expression.
                Please, be careful and consistent with your definitions!
                Example: prop_fun = 'A*k1'.
            delta: 'dict'
                It presents the discrete increase/reduction in copy number for each affected (product) species.
                Each dictionary (key, value) pair must be like ('str', 'int').
                Example: delta = {'A': -10, 'Z': 2}.
            equation: 'str', 'optional'
                    default = None
                It provides an equation (optional) for the new reaction.
                Example: equation = 'Reactants-k10->Products'.
                Note: the argument 'equation' is not compulsory, but it will be used in a future definiton of the 'BiochemStem' class.
        
        ########
        Returns
            self.reactions: 'dict'
            It returns a dictionary with an entry (biochemical reaction) added to the current system.
        
        """
        _temp = '{' + "'{0}': {0}, '{1}': {1}, '{2}': {2}".format('prop_fun', 'delta', 'equation') + '}'
        temp = eval(_temp)
        _consistent = self._consistency(temp, checkers = ['prop_fun', 'delta'], verbose = verbose)
        if not all(_consistent.values()):
            raise RuntimeError('The new reaction is wrong! Check it again!') # Stop!
        self.reactions[name] = temp
        if verbose:
            message = "We have successfully added '{}'!".format(name)
            print(message)
        self._assembled = False # Enforce flag
        return self
    
    def del_reaction(self, name):
        """It deletes a reaction from the current biochemical system based on the 'name' argument.
        
        ########
        Arguments
            name: 'str'
                We remove the reaction given by 'name' from the current biochemical system.
                It must be consistent with the argument 'name' provided when the method 'add_reaction' was invoked.
        
        ########
        Returns
            self.reactions: 'dict'
            It returns a dictionary with an entry (biochemical reaction) deleted from the current system.
        
        """
        self.reactions.pop(name)
        message = "We have successfully deleted '{}'!".format(name)
        print(message)
        self._assembled = False # Enforce flag
        return self
    
    def _delta_mat(self, rows, cols):
        delta_mat = np.full(shape = (len(rows), len(cols)), fill_value = np.nan, dtype = np.int32)
        it = np.nditer(op = delta_mat, flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = len(cols))
        for d in it:
            _i = divmod(it.iterindex, len(cols))
            i = _i[0]
            if i == 0:
                d[...] = [0] * len(cols) # d[...] = list(self.initial_state.values())
            else:
                _j = self.reactions[rows[i]]['delta']
                j = [(_j[key] if key in _j else 0) for key in cols.values()]
                d[...] = j
        return delta_mat
    
    def assemble(self, show = False):
        """It assembles our biochemical system; it makes it ready for simulation/analysis.
        
        ########
        Arguments
            show: 'bool'
                    default = True
                Do we want to see the assembled system?
                All the (biochemical) components will be shown in a cohesive way.
        
        ########
        Returns
            self.assembly: 'dict'
                Keys
                    'species': dictionary enumerating each species name.
                    'reactions': dictionary enumerating each reaction name.
                    'prop_funs': list containing propensity functions for all reactions.
                    'delta_mat': a matrix representing the delta for every reaction.
            It returns essential components for simulation/analysis.
        
        """
        species = dict(enumerate(self.initial_state)) # species = dict(zip(self.initial_state, range(len(self.initial_state))))
        reactions = dict(enumerate(['r0', *self.reactions])) # reactions = dict(zip(['r0', *self.reactions], range(len(self.reactions)+1)))
        prop_funs = ['0']
        prop_funs.extend([self.reactions[key]['prop_fun'] for key in self.reactions])
        delta_mat = self._delta_mat(rows = reactions, cols = species)
        self.assembly = {'species': species, 'reactions': reactions, 'prop_funs': prop_funs, 'delta_mat': delta_mat}
        self._assembled = True # Enforce flag
        if show:
            print(str(self))
        else:
            print("\n") # print(repr(self))
        return self
    

In [ ]:
# Example # (Class) BiochemStem

initial_state = {'A': 0}
rates = {'k1': 0.1, 'k2': 10}
a = BiochemStem(initial_state, rates)

In [ ]:
prop_fun = 'A*k1'
delta = {'A': -1}
a.add_reaction('r1', prop_fun, delta)

In [ ]:
prop_fun = 'k2'
delta = {'A': 1}
a.add_reaction('r2', prop_fun, delta)

In [ ]:
a.assemble()

In [ ]:
a.assembly

In [ ]:
_n = 'BiochemStem'
n = '#' * len(_n)
S = 'Species'
s = '-' * len(S)
t = list(initial_state)
I = 'Initial State'
i ='-' * len(I)
R = 'Rates'
_R = '-' * len(R)
r = a.rates
_t = list(initial_state.values())
m = a.stem['delta_mat'][1:]
_m = '-' * len('Delta/Reaction')
temp = f'{n}\n{_n}\n{n}\n\n\t\t{S}\n\t\t{s}\n{t}\n\n\t\t{I}\n\t\t{i}\n{_t}\n\n{R}\n{_R}\n\t{r}\n\nDelta/Reaction\n{_m}\n{m}'

In [ ]:
print(temp)

In [ ]:
_portrait = {'start': ('BiochemStem', '#'), 'mid': ([('Species', list(a.initial_state)), ('Initial State', list(a.initial_state.values()), ('Rates', a.rates)], '-'), 'final': ()}

In [ ]:
# Testa

initial_state = {'A': 0}
kb = 0.2
kf = 100
rates = {'kb': kb, 'kf': kf}
a = BiochemStem(initial_state, rates)

prop_fun = 'A*kb'
delta = {'A': -1}
a.add_reaction('r1', prop_fun, delta)

prop_fun = 'kf'
delta = {'A': 1}
a.add_reaction('r2', prop_fun, delta)

a.assemble()

testa = BiochemSimul(a, 10000, 1)
testa.meth_direct()
plt.plot(testa.epoch_mat[:, 0], testa.state_tor[:, 0, 0])
print(testa.epoch_mat[testa.state_tor[:, 0, 0] == (kf/(kb))/2, 0])

In [ ]:
# (Class) BiochemSimul

class BiochemSimul:
    
    """Class 'BiochemSimul' Illustration!
    It is an essential part of the 'BiochemAnalysis' class!
    
    ########
    Attributes
        
    
    ########
    Methods
        
    
    """
    
    def __init__(self, stem, steps, trajectories, seed = None):
        """
        
        
        """
        if not stem._assembled:
            raise RuntimeError('The biochemical system is not assembled! Check it again!') # Stop!
        self.stem = stem
        self.steps = steps
        self.trajectories = trajectories
        self.seed = seed
    
    def __repr__(self):
        portrait = f'<{self.__class__.__name__}({self.stem.__class__.__name__}, {self.steps}, {self.trajectories})>'
        return portrait
    
    def __str__(self):
        portrait = repr(self)
        return portrait
    
    def _ran_tor(self):
        self.ran_tor = np.random.default_rng(seed = self.seed).uniform(size = (self.steps, 2, self.trajectories))
        self.ran_tor[:, 0, :] = -np.log(self.ran_tor[:, 0, :])
        return self
    
    def _epoch_mat(self):
        self.epoch_mat = np.full(shape = (self.steps, self.trajectories), fill_value = np.nan)
        self.epoch_mat[0] = 0
        return self
    
    def _state_tor(self):
        self.state_tor = np.zeros(shape = (self.steps, len(self.stem.assembly['species']), self.trajectories), dtype = np.uint32)
        return self
    
    def _iteration(self, t):
        self.state_tor[0, :, t] = list(self.stem.initial_state.values())
        it = np.nditer(op = self.state_tor[1:, :, t], flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.state_tor.shape[1])
        # it = np.nditer(op = self.state_tor[..., t], flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.state_tor.shape[1])
        return it
    
    def _tracker(self, it):
        _i = divmod(it.iterindex, self.state_tor.shape[1])
        i = _i[0] + 1
        return i
    
    def _pre(self, t):
        pre = self.state_tor[0, :, t]
        return pre
    
    def _alps(self, step, trajectory):
        for i, j in self.stem.rates.items():
            express = f'{i} = {j}'
            exec(express)
        for _ in self.stem.assembly['species']:
            key = self.stem.assembly['species'][_]
            value = self.state_tor[step - 1, _, trajectory]
            express = f'{key} = {value}'
            exec(express)
        _alps = []
        for alp in self.stem.assembly['prop_funs']:
            _alps.append(eval(alp))
        alps = np.zeros(len(_alps))
        alps[0] = np.sum(_alps)
        alps[1:] = np.cumsum(_alps[1:]) / alps[0]
        return alps
    
    def _refresh_epoch(self, step, trajectory, alps):
        tau = self.ran_tor[step, 0, trajectory] / alps[0]
        self.epoch_mat[step, trajectory] = self.epoch_mat[step - 1, trajectory] + tau
        return self
    
    def _refresh_delta(self, step, trajectory, alps):
        temp = np.argwhere(self.ran_tor[step, 1, trajectory] < alps[1:])
        if not any(temp):
            disc = len(alps) - 1
        else:
            disc = temp[0, 0] + 1
        delta = self.stem.assembly['delta_mat'][disc, :]
        return delta
    
    def meth_direct(self):
        self._ran_tor()
        self._epoch_mat()
        self._state_tor()
        for trajectory in range(self.trajectories):
            state_mat = self._iteration(trajectory)
            pre = self._pre(trajectory)
            for state in state_mat:
                step = self._tracker(state_mat) # [1, self.steps - 1]
                alps = self._alps(step, trajectory)
                self._refresh_epoch(step, trajectory, alps)
                delta = self._refresh_delta(step, trajectory, alps)
                state[...] = pre + delta
                pre = state
        return self
    

In [ ]:
# (Class) BiochemSimul (Copy!) # No seed!

class BiochemSimul:
    
    """Class 'BiochemSimul' Illustration!
    It is an essential part of the 'BiochemAnalysis' class!
    
    ########
    Attributes
        
    
    ########
    Methods
        
    
    """
    
    def __init__(self, stem, steps, trajectories):
        """
        
        
        """
        if not stem._assembled:
            raise RuntimeError('The biochemical system is not assembled! Check it again!') # Stop!
        self.stem = stem
        self.steps = steps
        self.trajectories = trajectories
    
    def __repr__(self):
        portrait = f'<{self.__class__.__name__}({self.stem.__class__.__name__}, {self.steps}, {self.trajectories})>'
        return portrait
    
    def __str__(self):
        portrait = repr(self)
        return portrait
    
    def _ran_tor(self):
        self.ran_tor = np.random.default_rng().uniform(size = (self.steps, 2, self.trajectories))
        self.ran_tor[:, 0, :] = -np.log(self.ran_tor[:, 0, :])
        return self
    
    def _epoch_mat(self):
        self.epoch_mat = np.full(shape = (self.steps, self.trajectories), fill_value = np.nan)
        self.epoch_mat[0] = 0
        return self
    
    def _state_tor(self):
        self.state_tor = np.zeros(shape = (self.steps, len(self.stem.assembly['species']), self.trajectories), dtype = np.uint32)
        return self
    
    def _iteration(self, t):
        self.state_tor[0, :, t] = list(self.stem.initial_state.values())
        it = np.nditer(op = self.state_tor[1:, :, t], flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.state_tor.shape[1])
        # it = np.nditer(op = self.state_tor[..., t], flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.state_tor.shape[1])
        return it
    
    def _tracker(self, it):
        _i = divmod(it.iterindex, self.state_tor.shape[1])
        i = _i[0] + 1
        return i
    
    def _pre(self, t):
        pre = self.state_tor[0, :, t]
        return pre
    
    def _alps(self, step, trajectory):
        for i, j in self.stem.rates.items():
            express = f'{i} = {j}'
            exec(express)
        for _ in self.stem.assembly['species']:
            key = self.stem.assembly['species'][_]
            value = self.state_tor[step - 1, _, trajectory]
            express = f'{key} = {value}'
            exec(express)
        _alps = []
        for alp in self.stem.assembly['prop_funs']:
            _alps.append(eval(alp))
        alps = np.zeros(len(_alps))
        alps[0] = np.sum(_alps)
        alps[1:] = np.cumsum(_alps[1:]) / alps[0]
        return alps
    
    def _refresh_epoch(self, step, trajectory, alps):
        tau = self.ran_tor[step, 0, trajectory] / alps[0]
        self.epoch_mat[step, trajectory] = self.epoch_mat[step - 1, trajectory] + tau
        return self
    
    def _refresh_delta(self, step, trajectory, alps):
        temp = np.argwhere(self.ran_tor[step, 1, trajectory] < alps[1:])
        if not any(temp):
            disc = len(alps) - 1
        else:
            disc = temp[0, 0] + 1
        delta = self.stem.assembly['delta_mat'][disc, :]
        return delta
    
    def meth_direct(self):
        self._ran_tor()
        self._epoch_mat()
        self._state_tor()
        for trajectory in range(self.trajectories):
            state_mat = self._iteration(trajectory)
            pre = self._pre(trajectory)
            for state in state_mat:
                step = self._tracker(state_mat) # [1, self.steps - 1]
                alps = self._alps(step, trajectory)
                self._refresh_epoch(step, trajectory, alps)
                delta = self._refresh_delta(step, trajectory, alps)
                state[...] = pre + delta
                pre = state
        return self
    

In [ ]:
# (Class) _BiochemSimul # All trajectories at once!

class _BiochemSimul:
    
    """Class 'BiochemSimul' Illustration!
    It is an essential part of the 'BiochemAnalysis' class!
    
    ########
    Attributes
        
    
    ########
    Methods
        
    
    """
    
    def __init__(self, stem, steps, trajectories, seed = None):
        """
        
        
        """
        if not stem._assembled:
            raise RuntimeError('The biochemical system is not assembled! Check it again!') # Stop!
        self.stem = stem
        self.steps = steps
        self.trajectories = trajectories
        self.seed = seed
    
    def __repr__(self):
        portrait = f'<{self.__class__.__name__}({self.stem.__class__.__name__}, {self.steps}, {self.trajectories})>'
        return portrait
    
    def __str__(self):
        portrait = repr(self)
        return portrait
    
    def _ran_tor(self):
        self.ran_tor = np.random.default_rng(seed = self.seed).uniform(size = (self.steps, 2, self.trajectories))
        self.ran_tor[:, 0, :] = -np.log(self.ran_tor[:, 0, :])
        return self
    
    def _epoch_mat(self):
        self.epoch_mat = np.full(shape = (self.steps, self.trajectories), fill_value = np.nan)
        self.epoch_mat[0] = 0
        return self
    
    def _state_tor(self):
        self.state_tor = np.zeros(shape = (self.steps, len(self.stem.assembly['species']), self.trajectories), dtype = np.uint32)
        return self
    
    def _iteration(self):
        for t in range(self.state_tor.shape[2]):
            self.state_tor[0, :, t] = list(self.stem.initial_state.values())
        self.it = np.nditer(op = self.state_tor[1:, ...], flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.state_tor.shape[1]*self.state_tor.shape[2])
        # it = np.nditer(op = self.state_tor[..., t], flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.state_tor.shape[1])
        return self
    
    def _tracker(self):
        _i = divmod(self.it.iterindex, self.state_tor.shape[1]*self.state_tor.shape[2])
        i = _i[0] + 1
        return i
    
    def _pre(self):
        pre = self.state_tor[0, ...]
        return pre
    
    def _alps(self, step):
        for i, j in self.stem.rates.items():
            express = f'{i} = {j}'
            exec(express)
        for _ in self.stem.assembly['species']:
            key = self.stem.assembly['species'][_]
            value = self.state_tor[step - 1, _, :]
            express = f'{key} = value'
            exec(express)
        alp_mat = np.zeros((len(self.stem.assembly['prop_funs']), self.trajectories))
        test = alp_mat
        it = np.nditer(op = alp_mat, flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.trajectories)
        for alp in it:
            tracker = divmod(it.iterindex, self.trajectories)[0]
            prop_fun = self.stem.assembly['prop_funs'][tracker]
            alp[...] = eval(prop_fun)
        alp_mat[0] = np.sum(alp_mat, axis = 0)
        alp_mat[1:] = np.cumsum(alp_mat[1:], axis = 0) / alp_mat[0]
        return alp_mat
    
    def _refresh_epoch(self, step, alps):
        tau = self.ran_tor[step, 0, :] / alps[0]
        self.epoch_mat[step, :] = self.epoch_mat[step - 1, :] + tau
        return self

    def _refresh_delta(self, step, alps):
        temp = np.where(self.ran_tor[step, 1, :] <= alps[1:, :], self.stem.assembly['delta_mat'][1:], 0)
        delta = np.apply_along_axis(lambda x: x[x.nonzero()][0], 0, temp)
        return delta    
    
    def meth_direct(self):
        self._ran_tor()
        self._epoch_mat()
        self._state_tor()
        self._iteration()
        pre = self._pre()
        for state in self.it:
            step = self._tracker() # [1, self.steps - 1]
            #print('\n\n', step)
            #print('Pre\t', pre, '\n')
            alps = self._alps(step)
            #print(alps)
            self._refresh_epoch(step, alps)
            #print(self.epoch_mat[step, :])
            delta = self._refresh_delta(step, alps)
            #print(delta)
            #print(self.ran_tor[step, 1, :])
            state[...] = pre + delta
            pre = state
            #print(pre)
        return self
    

In [ ]:
# (Test ~ Class) BiochemSimul

class BiochemSimul_T:
    
    """Class 'BiochemSimul' Illustration!
    It is an essential part of the 'BiochemAnalysis' class!
    
    ########
    Attributes
        
    
    ########
    Methods
        
    
    """
    
    def __init__(self, stem, steps, trajectories, seed = None):
        """
        
        
        """
        if not stem._assembled:
            raise RuntimeError('The biochemical system is not assembled! Check it again!') # Stop!
        self.stem = stem
        self.steps = steps
        self.trajectories = trajectories
        self.seed = seed
    
    def __repr__(self):
        portrait = f'<{self.__class__.__name__}({self.stem.__class__.__name__}, {self.steps}, {self.trajectories})>'
        return portrait
    
    def __str__(self):
        portrait = repr(self)
        return portrait
    
    def _ran_tor(self):
        self.ran_tor = np.random.default_rng(seed = self.seed).uniform(size = (self.steps, 2, self.trajectories))
        self.ran_tor[:, 0, :] = -np.log(self.ran_tor[:, 0, :])
        return self
    
    def _epoch_mat(self):
        self.epoch_mat = np.full(shape = (self.steps, self.trajectories), fill_value = np.nan)
        self.epoch_mat[0] = 0
        return self
    
    def _state_tor(self):
        self.state_tor = np.zeros(shape = (self.steps, len(self.stem.assembly['species']), self.trajectories), dtype = np.uint32)
        return self
    
    def _iteration(self, t):
        self.state_tor[0, :, t] = list(self.stem.initial_state.values())
        it = np.nditer(op = self.state_tor[1:, :, t], flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.state_tor.shape[1])
        # it = np.nditer(op = self.state_tor[..., t], flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.state_tor.shape[1])
        return it
    
    def _tracker(self, it):
        _i = divmod(it.iterindex, self.state_tor.shape[1])
        i = _i[0] + 1
        return i
    
    def _pre(self, t):
        pre = self.state_tor[0, :, t]
        return pre
    
    def _alps(self, step, trajectory):
        for i, j in self.stem.rates.items():
            express = f'{i} = {j}'
            exec(express)
        for _ in self.stem.assembly['species']:
            key = self.stem.assembly['species'][_]
            value = self.state_tor[step - 1, _, trajectory]
            express = f'{key} = {value}'
            exec(express)
        _alps = []
        for alp in self.stem.assembly['prop_funs']:
            _alps.append(eval(alp))
        alps = np.zeros(len(_alps))
        alps[0] = np.sum(_alps)
        alps[1:] = np.cumsum(_alps[1:]) / alps[0]
        return alps
    
    def _refresh_epoch(self, step, trajectory, alps):
        tau = self.ran_tor[step, 0, trajectory] / alps[0]
        self.epoch_mat[step, trajectory] = self.epoch_mat[step - 1, trajectory] + tau
        return self
    
    def _refresh_delta(self, step, trajectory, alps):
        temp = np.argwhere(self.ran_tor[step, 1, trajectory] < alps[1:])
        if not any(temp):
            disc = len(alps) - 1
        else:
            disc = temp[0, 0] + 1
        delta = self.stem.assembly['delta_mat'][disc, :]
        return delta
    
    def meth_direct(self):
        self._ran_tor()
        self._epoch_mat()
        self._state_tor()
        for trajectory in range(self.trajectories):
            state_mat = self._iteration(trajectory)
            pre = self._pre(trajectory)
            for state in state_mat:
                step = self._tracker(state_mat) # [1, self.steps - 1]
                print('\n\n', step)
                # print('Pre\t', pre, '\n')
                alps = self._alps(step, trajectory)
                # print(alps)
                self._refresh_epoch(step, trajectory, alps)
                # print(self.epoch_mat[step, :])
                delta = self._refresh_delta(step, trajectory, alps)
                print(delta)
                print(self.ran_tor[step, 1, trajectory])
                state[...] = pre + delta
                pre = state
                # print(pre)
        return self
    

In [ ]:
# Test Class [_BiochemSimul ~ BiochemSimul]

In [ ]:
initial_state = {'A': 100}
rates = {'k1': 0.1, 'k2': 10}
a = BiochemStem(initial_state, rates)

In [ ]:
prop_fun = 'A*k1'
delta = {'A': -1}
a.add_reaction('r1', prop_fun, delta)

In [ ]:
prop_fun = 'k2'
delta = {'A': 1}
a.add_reaction('r2', prop_fun, delta)

In [ ]:
a.assemble()

In [ ]:
a.assembly

In [ ]:
z0 = _BiochemSimul(a, 10, 2, 25)
z0.meth_direct()

def _alps0(self, step):
    for i, j in self.stem.rates.items():
        express = f'{i} = {j}'
        exec(express)
    for _ in self.stem.assembly['species']:
        key = self.stem.assembly['species'][_]
        value = self.state_tor[step - 1, _, :]
        express = f'{key} = value'
        exec(express)
    alp_mat = np.zeros((len(self.stem.assembly['prop_funs']), self.trajectories))
    test = alp_mat
    it = np.nditer(op = alp_mat, flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = self.trajectories)
    for alp in it:
        tracker = divmod(it.iterindex, self.trajectories)[0]
        prop_fun = self.stem.assembly['prop_funs'][tracker]
        alp[...] = eval(prop_fun)
    alp_mat[0] = np.sum(alp_mat, axis = 0)
    alp_mat[1:] = np.cumsum(alp_mat[1:], axis = 0) / alp_mat[0]
    return alp_mat

def _refresh_epoch0(self, step, alps):
    tau = self.ran_tor[step, 0, :] / alps[0]
    self.epoch_mat[step, :] = self.epoch_mat[step - 1, :] + tau
    return self

def _refresh_delta0(self, step, alps):
    temp = np.where(self.ran_tor[step, 1, :] <= alps[1:, :], self.stem.assembly['delta_mat'][1:], 0)
    delta = np.apply_along_axis(lambda _: _[_.nonzero()][0], 0, temp)
    return delta

In [ ]:
step = 3
t0 = _alps0(z0, step)
print(t0)
t1 = _refresh_epoch0(z0, step, t0)
print(t1.epoch_mat[1, :])
t2 = _refresh_delta0(z0, step, t0)
print(t2)

In [ ]:
self.stem.assembly['delta_mat']

In [ ]:
z1 = BiochemSimul_T(a, 10, 2, 25)
z1.meth_direct()

def _alps1(self, step, trajectory):
        for i, j in self.stem.rates.items():
            express = f'{i} = {j}'
            exec(express)
        for _ in self.stem.assembly['species']:
            key = self.stem.assembly['species'][_]
            value = self.state_tor[step - 1, _, trajectory]
            express = f'{key} = {value}'
            exec(express)
        _alps = []
        for alp in self.stem.assembly['prop_funs']:
            _alps.append(eval(alp))
        alps = np.zeros(len(_alps))
        alps[0] = np.sum(_alps)
        alps[1:] = np.cumsum(_alps[1:]) / alps[0]
        return alps

def _refresh_epoch1(self, step, trajectory, alps):
    tau = self.ran_tor[step, 0, trajectory] / alps[0]
    self.epoch_mat[step, trajectory] = self.epoch_mat[step - 1, trajectory] + tau
    return self
    
def _refresh_delta1(self, step, trajectory, alps):
    temp = np.argwhere(self.ran_tor[step, 1, trajectory] < alps[1:])
    if not any(temp):
        disc = len(alps) - 1
    else:
        disc = temp[0, 0] + 1
    delta = self.stem.assembly['delta_mat'][disc, :]
    return delta

In [ ]:
step = 3
t0 = _alps1(z1, step, 0)
print(t0)
t1 = _refresh_epoch1(z1, step, 0, t0)
print(t1.epoch_mat[1, :])
t2 = _refresh_delta1(z1, step, 0, t0)
print(t2)

In [ ]:
self = z1
step = 3
alps = t0
trajectory = 0
temp = np.argwhere(self.ran_tor[step, 1, trajectory] < alps[1:])

In [ ]:
self.ran_tor[step, 1, trajectory]

In [ ]:
alps[1:]

In [ ]:
np.argwhere(self.ran_tor[step, 1, trajectory] < alps[1:])

In [ ]:
# We have to check the delta fun and fix it!

In [ ]:
# Check the following lines of code, because I don't remember the reason why I wrote them

In [ ]:
state_tor = np.array(list(range(10*5*2))).reshape((10, 5, 2))

In [ ]:
for t in [0, 1]:
    state_tor[0, :, t] = list(range(-5, 0))
it = np.nditer(op = state_tor[1:, ...], flags = ['external_loop', 'buffered'], op_flags = ['readwrite'], order = 'C', buffersize = state_tor.shape[1]*state_tor.shape[2])
for e in it:
    _ = e[...] % 2
    print(_)

In [ ]:
state_tor[0, ...]

In [ ]:
# Performance [Time] Comparison

In [ ]:
%%time
z0 = _BiochemSimul(a, 100000, 100, 25)
z0.meth_direct()

In [ ]:
%%time
z1 = BiochemSimul(a, 10000, 100, 25)
z1.meth_direct()

In [ ]:
for t in range(z0.trajectories):
    x = z0.epoch_mat[:, t]
    y = z0.state_tor[..., t]
    plt.plot(x, y)
plt.show()
for t in range(z1.trajectories):
    x = z1.epoch_mat[:, t]
    y = z1.state_tor[..., t]
    plt.plot(x, y)

In [ ]:
np.all((z0.state_tor == z1.state_tor))

In [ ]:
# Example # Control Transcription

# P_ := Promoter Binding Site _
# TF := Transcription Factor
# mRNA

nP = 4 # nP = 4 ~ nTF = 1000
nTF = 940
initial_state = {f'P{_}': (1 if _ == 0 else 0) for _ in range(nP+1)}
initial_state.update({'TF': nTF, 'mRNA': 0})

# rates = {'kf': 0.1, 'kb': 1}
u = 5
k0 = 100
pre = 2000 # 1000 # After to 20
rates = {'kf': 0.01, 'kd': pre*0.0017, 'km': pre*0.17/1} # rates = {'kf': 0.5, 'kd': 0.005}
rates.update({f'kb{_}': (k0 if _ == 1 else k0*pow(1/u, _-1)) for _ in range(1, nP+1)})

e = BiochemStem(initial_state, rates)
rates

In [ ]:
[1/i for i in rates.values()]

In [ ]:
# Forward

for _ in range(nP):
    prop_fun = f'P{_}*TF*kf'
    delta = {f'P{_}': -1, f'P{_+1}': 1, 'TF': -1}
    e.add_reaction(f'P{_+1}f', prop_fun, delta)

e.add_reaction('mRNAf', f'P{nP}*km', {'mRNA': 1})

In [ ]:
# Backward

for _ in range(nP):
    prop_fun = f'P{_+1}*kb{_+1}'
    delta = {f'P{_}': 1, f'P{_+1}': -1, 'TF': 1}
    e.add_reaction(f'P{_}b', prop_fun, delta)

e.add_reaction('mRNAb', f'mRNA*kd', {'mRNA': -1})

In [ ]:
rates

In [ ]:
e.assemble()
e.assembly

In [ ]:
%%time
steps = 200000
trajectories = 1
w = BiochemSimul(e, steps, trajectories)
w.meth_direct()

In [ ]:
# This has to be fixed; it's returning wrong estimates for the time average!
def time_ave(w, j):
    observe = w.state_tor[:, j, :]
    inters = np.diff(w.epoch_mat[:, :], axis = 0)
    zero = np.where(observe[:-1] == 0, inters, np.nan)
    one = np.where(observe[:-1] == 1, inters, np.nan)
    s0 = np.nanmean(zero, axis = 0)
    s1 = np.nanmean(one, axis = 0)
    return (s0, s1)

j = 4
z, o = time_ave(w, j)
print(f'Mean Interreaction Time for Zero TF Bound on P{j}\n', z, '\n')
print(f'Mean Interreaction Time for One TF Bound on P{j}\n', o)

In [ ]:
_y = w.state_tor[:, j, 0]
_x = w.epoch_mat[:, 0]

dx = np.diff(_x)
dy = np.diff(_y)

#plt.plot(_x[1:], _y[1:], ls = '-', marker = '+')
#plt.plot(_x[1:], dy, ls = '-', marker = '+')

where = np.insert(np.where(dy != 0, True, False), 0, True)
pro = np.diff(np.append(_x[where], _x[-1]))
what = _y[where]
_what = np.unique(what)
lis = []

for i in _what:
    _ = np.nanmean(np.where(what == i, pro, np.nan))
    print(i, _)


In [ ]:
print(rates)
print(nTF, '\n')
print(1/(nTF*rates['kf']))
print(1/rates['kb1'])

In [ ]:
_s = 90000
s = 100000 # s = steps
for t in range(w.trajectories):
    x = w.epoch_mat[_s:s, t]
    mRNA = w.state_tor[_s:s, -1, t]
    nTFcurrent = w.state_tor[_s:s, 5, t]
    nTFbound = nTF - nTFcurrent
    P4 = w.state_tor[_s:s, 4, t]
    #y = nTFbound
    #y = mRNA
    y = P4
    plt.plot(x, mRNA/100, ls = '-', marker = '')
    #plt.plot(x, mRNA, ls = '-', marker = '')
    plt.plot(x, y, ls = '-', marker = '')
    #plt.plot(x, [np.mean(y)-np.std(y)]*len(x))
    #plt.plot(x, [np.mean(y)]*len(x))
    #plt.plot(x, [time_ave(w, 4)[0]]*len(x))
    #plt.plot(x, [np.mean(y)+np.std(y)]*len(x))
    plt.show()
    # plt.savefig('A')
#np.mean(y)

In [ ]:
# Is this equivalent to equidistant fun? Yes! :D

# Original data

_x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 40, 50, 100])
# _y = np.array([3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
_y = np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
if len(_x) != len(_y):
    raise "Oops"
print(_x)
print(np.mean(_y), np.var(_y), '\n')
plt.plot(_x, _y, ls = '-', marker = '+')
plt.show()

# Interpolated data

_ix = np.linspace(0, _x[-1], 1000*len(_x))
_fun = interpolate.interp1d(x = _x, y = _y, kind = 0)
_iy = _fun(_ix)
print('\n', _ix)
print(np.mean(_iy), np.var(_iy))
plt.plot(_ix, _iy, ls = '-', marker = '+')

In [ ]:
# This has to be transformed into a function!

dx = np.diff(_x)
dy = np.diff(_y)

#plt.plot(_x[1:], _y[1:], ls = '-', marker = '+')
#plt.plot(_x[1:], dy, ls = '-', marker = '+')

where = np.insert(np.where(dy != 0, True, False), 0, True)
pro = np.diff(np.append(_x[where], _x[-1]))
what = _y[where]
_what = np.unique(what)
lis = []

for i in _what:
    _ = np.sum(np.where(what == i, pro, 0))/_x[-1]
    lis.append(_)

ave = np.average(a = _what, weights = np.array(lis))
var = np.average(a = np.power(_what, 2), weights = np.array(lis)) - pow(ave, 2)
print(ave, var)

In [ ]:
# This doesn't shows the mean interarrival times!

for i in _what:
    _ = np.sum(np.where(what == i, pro, 0))/_x[-1]
    print(i, _)

In [ ]:
# Interpolated trajectory

t = 0

x = w.epoch_mat[:, t]
y = w.state_tor[:, 4, t]

_ix = np.linspace(0, x[-1], 100*len(x))
_fun = interpolate.interp1d(x = x, y = y, kind = 0)
_iy = _fun(_ix)
print('Ave', np.mean(_iy), '\t', 'Var', np.var(_iy))
plt.plot(_ix, _iy, ls = '-')

In [ ]:
# The following cell should be transformed into a function!

In [ ]:
elements = np.unique(y)
times = np.diff(w.epoch_mat[:, t])
lis = []

for i in elements:
    _ = np.sum(np.where(y[:-1] == i, times, 0))/np.max(w.epoch_mat[:, t])
    lis.append(_)

ave = np.average(a = elements, weights = np.array(lis))
var = np.average(a = np.power(elements, 2), weights = np.array(lis)) - pow(ave, 2)
print('Ave', ave, '\t', 'Var', var, '\t', 'Std', np.sqrt(var))

plt.plot(elements, lis, marker = '+')
plt.vlines([ave, ave - np.sqrt(var), ave + np.sqrt(var)], 0, max(lis))

In [ ]:
# Simulate me! (Old)

In [ ]:
class Test:
    w = 'Zero'
    def _a():
        exec("global x\nx = 'Test'")
        print(locals())
        return None
    def _z():
        print(x)
        print(w)
        return None

In [ ]:
# nP = 4
NTF = 2000
# rates = {'kf': 0.1, 'kb': 1}
steps = 20000 # steps = 200000
trajectories = 1
jumps = 100
species = 'mRNA'

def time_ave(w, j):
    observe = w.state_tor[:, j, 0]
    inters = np.diff(w.epoch_mat[:, 0])
    zero = np.where(observe[:-1] == 0, inters, 0)
    one = np.where(observe[:-1] == 1, inters, 0)
    s0 = np.sum(zero)/np.sum(inters)
    s1 = np.sum(one)/np.sum(inters)
    s_ = np.mean(observe)
    print(s0)
    print(s1)
    print(s0 + s1)
    print(s_)
    return (s1, s_)

def hill_fun():
    lis = list(range(0, 2100, 100))
    ret = {x: pow(x, 4)/(pow(940, 4) + pow(x, 4)) for x in lis}
    return ret

def simulator(nP, NTF, rates, steps, trajectories, jumps, species):
    
    d = {} # A container for the activation number!
    
    for nTF in range(0, NTF+1, jumps):
        # if nTF == 0:
        #     nTF = 1
        initial_state = {f'P{_}': (1 if _ == 0 else 0) for _ in range(nP+1)}
        initial_state.update({'TF': nTF, 'mRNA': 0})
        e = BiochemStem(initial_state, rates)
        
        # Forward

        for _ in range(nP):
            prop_fun = f'P{_}*TF*kf'
            delta = {f'P{_}': -1, f'P{_+1}': 1, 'TF': -1}
            e.add_reaction(f'P{_+1}f', prop_fun, delta)

        e.add_reaction('mRNAf', f'P{nP}*km', {'mRNA': 1})
        
        # Backward

        for _ in range(nP):
            prop_fun = f'P{_+1}*kb{_+1}'
            delta = {f'P{_}': 1, f'P{_+1}': -1, 'TF': 1}
            e.add_reaction(f'P{_}b', prop_fun, delta)

        e.add_reaction('mRNAb', f'mRNA*kd', {'mRNA': -1})
        
        e.assemble()
        # e.assembly
        
        w = BiochemSimul(e, steps, trajectories)
        w.meth_direct()
        
        _j = list(w.stem.assembly['species'].values())
        j = _j.index(species)
        #d.update({nTF: np.mean(w.state_tor[:, j, :])})
        d.update({nTF: time_ave(w, j)})
        
        print(nTF)
        #nTFcurrent = w.state_tor[:, j, :]
        #nTFbound = nTF - nTFcurrent
        #d.update({nTF: np1.mean(nTFbound)})
    
    return d

In [ ]:
# Old

d = simulator(nP, NTF, rates, steps, trajectories, jumps, species)
print(d)

In [ ]:
# Old

_keys = list(d.keys())
_keys.sort()
_values = list(d.values())
_values = [(0 if i[1] == 0.0 else i[0]) for i in _values]
_values.sort()

h = hill_fun()
plt.plot(list(h.keys()), list(h.values()), color = 'orange')

plt.plot(_keys, _values, color = 'green', marker = '+')
plt.hlines([0, 0.5, 1], 0, 2000, colors = 'lightgray', linestyles = 'dashed')
#plt.hlines([0, 0.5, d[200], 3.5, 4], 0, 1000, colors = ['lightgray', 'lightgray', 'gray','lightgray', 'lightgray'], linestyles = 'dashed')
plt.vlines(940, 0, 1, colors = 'gray', linestyles = 'dashed')
#plt.savefig('0-100-2000_P4.jpeg', dpi = 200, quality = 95)
#plt.show()

#plt.plot(keys, values, color = 'orange', marker = '+')
#plt.hlines(values[4], 0, 1, colors = 'gray', linestyles = 'dashed')
#plt.vlines(keys[4], 0, 1, colors = 'gray', linestyles = 'dashed')
# plt.savefig('Z.jpeg', dpi = 200, quality = 95)

In [ ]:
# Simulate me! (New)

In [ ]:
# Careful! Do not run this one!

# nP = 4
NTF = 2000
# rates = {'kf': 0.1, 'kb': 1}
steps = 10000 # steps = 200000
trajectories = 2
jumps = 100
species = 'P4'

def time_ave(w, j, species = ''):
    if species == 'mRNA':
        observe = pd.DataFrame(w.state_tor[:, j, :])
        _ = w.state_tor.shape[0]
        s = observe.rolling(int(_/100)).mean().mean()
        s1 = np.array(s)
    else:
        observe = w.state_tor[:, j, :]
        inters = np.diff(w.epoch_mat, axis = 0)
        zero = np.where(observe[:-1] == 0, inters, 0)
        one = np.where(observe[:-1] == 1, inters, 0)
        s0 = np.sum(zero, axis = 0)/np.sum(inters, axis = 0)
        s1 = np.sum(one, axis = 0)/np.sum(inters, axis = 0)
        s_ = np.mean(observe, axis = 0)
    # print(s0)
    print('Ave\t', s1)
    # print(s0 + s1)
    # print(s_)
    return s1

def naive_time_variance(w, j, species = ''):
    if species == 'mRNA':
        s1 = 0
    else:
        observe = w.state_tor[:, j, :]
        s1 = np.std(observe, axis = 0)
    print('NStd\t', s1)
    return s1

def time_variance(w, j, species = ''):
    observe = pd.DataFrame(w.state_tor[:, j, :])
    _ = w.state_tor.shape[0]
    s = observe.rolling(int(_/100)).std().mean()
    s1 = np.array(s)
    print('Std', s1)
    return s1

def hill_fun():
    lis = list(range(0, 2100, 100))
    ret = {x: pow(x, 4)/(pow(940, 4) + pow(x, 4)) for x in lis}
    return ret

def simulator(nP, NTF, rates, steps, trajectories, jumps, species):
    
    d = {} # A container for the activation number!
    
    for nTF in range(0, NTF+1, jumps):
        # if nTF == 0:
        #     nTF = 1
        initial_state = {f'P{_}': (1 if _ == 0 else 0) for _ in range(nP+1)}
        initial_state.update({'TF': nTF, 'mRNA': 0})
        e = BiochemStem(initial_state, rates)
        
        # Forward

        for _ in range(nP):
            prop_fun = f'P{_}*TF*kf'
            delta = {f'P{_}': -1, f'P{_+1}': 1, 'TF': -1}
            e.add_reaction(f'P{_+1}f', prop_fun, delta)

        e.add_reaction('mRNAf', f'P{nP}*km', {'mRNA': 1})
        
        # Backward

        for _ in range(nP):
            prop_fun = f'P{_+1}*kb{_+1}'
            delta = {f'P{_}': 1, f'P{_+1}': -1, 'TF': 1}
            e.add_reaction(f'P{_}b', prop_fun, delta)

        e.add_reaction('mRNAb', f'mRNA*kd', {'mRNA': -1})
        
        e.assemble()
        # e.assembly
        
        w = BiochemSimul(e, steps, trajectories)
        w.meth_direct()
        
        _j = list(w.stem.assembly['species'].values())
        j = _j.index(species)
        #d.update({nTF: np.mean(w.state_tor[:, j, :])})
        d.update({nTF: {'Ave': time_ave(w, j, species), 'NStd': naive_time_variance(w, j), 'Std': time_variance(w,j)}})
        
        print(nTF)
        #nTFcurrent = w.state_tor[:, j, :]
        #nTFbound = nTF - nTFcurrent
        #d.update({nTF: np1.mean(nTFbound)})
    
    return d

In [ ]:
d = simulator(nP, NTF, rates, steps, trajectories, jumps, species)
print(d)

In [ ]:
_NTF = list(d.keys())
Ave = np.array([np.mean(d[i]['Ave']) for i in _NTF])
Std = np.array([np.mean(d[i]['Std']) for i in _NTF])
Ave[0] = 0
Std[0] = 0

h = hill_fun()
#plt.plot(list(h.keys()), list(h.values()), color = 'orange')

plt.plot(_NTF, Ave, color = 'green', marker = '+')
plt.plot(_NTF, Ave + Std, color = 'pink', marker = '.')
plt.plot(_NTF, Ave - Std, color = 'pink', marker = '.')
plt.hlines([0, 0.5, 1], 0, 2000, colors = 'lightgray', linestyles = 'dashed')
plt.vlines(940, -0.25, 1.25, colors = 'gray', linestyles = 'dashed')
#plt.savefig('0-100-2000_P4.jpeg', dpi = 200, quality = 95)

In [ ]:
plt.plot(_NTF, Std / Ave)
#plt.savefig('0-100-2000_mRNA_CV.jpeg', dpi = 200, quality = 95)
plt.show()
plt.plot(_NTF, np.power(Std, 2) / Ave)
plt.show()
#plt.savefig('0-100-2000_mRNA_Fan.jpeg', dpi = 200, quality = 95)
plt.scatter(Ave, Std)
plt.savefig('0-100-2000_mRNA_Ave_Std.jpeg', dpi = 200, quality = 95)

In [ ]:
# Careful! This has a new version of the variance estimation function!

# nP = 4
NTF = 2000 # 3000
# rates = {'kf': 0.1, 'kb': 1}
steps = 200000 # steps = 50000
trajectories = 1
jumps = 100
species = 'mRNA'

pre = 1500 # 2000 # 20
rates = {'kf': 0.01, 'kd': pre*0.0017, 'km': pre*0.17/1} # rates = {'kf': 0.5, 'kd': 0.005}
rates.update({f'kb{_}': (k0 if _ == 1 else k0*pow(1/u, _-1)) for _ in range(1, nP+1)})

def time_ave(w, j, species = ''):
    if species == 'mRNA':
        observe = pd.DataFrame(w.state_tor[:, j, :])
        _ = w.state_tor.shape[0]
        s = observe.rolling(int(_/100)).mean().mean()
        s1 = np.array(s)
    else:
        observe = w.state_tor[:, j, :]
        inters = np.diff(w.epoch_mat, axis = 0)
        zero = np.where(observe[:-1] == 0, inters, 0)
        one = np.where(observe[:-1] == 1, inters, 0)
        s0 = np.sum(zero, axis = 0)/np.sum(inters, axis = 0)
        s1 = np.sum(one, axis = 0)/np.sum(inters, axis = 0)
        s_ = np.mean(observe, axis = 0)
    # print(s0)
    print('Ave\t', s1)
    # print(s0 + s1)
    # print(s_)
    return s1

def naive_time_variance(w, j, species = ''):
    if species == 'mRNA':
        s1 = 0
    else:
        observe = w.state_tor[:, j, :]
        s1 = np.std(observe, axis = 0)
    print('NStd\t', s1)
    return s1

def time_variance(w, j, species = ''):
    observe = pd.DataFrame(w.state_tor[:, j, :])
    _ = w.state_tor.shape[0]
    s = observe.rolling(int(_/100)).std().mean()
    s1 = np.array(s)
    print('Std', s1)
    return s1

def variance(w, j):
    t = 0
    y = w.state_tor[:, j, t] # Only one trajectory for now!
    elements = np.unique(y)
    times = np.diff(w.epoch_mat[:, t])
    lis = []
    
    for i in elements:
        _ = np.sum(np.where(y[:-1] == i, times, 0))/np.max(w.epoch_mat[:, t])
        lis.append(_)

    ave = np.average(a = elements, weights = np.array(lis))
    var = np.average(a = np.power(elements, 2), weights = np.array(lis)) - pow(ave, 2)
    #print('Ave', ave, '\t', 'Var', var, '\t', 'Std', np.sqrt(var))
    
    #plt.plot(elements, lis, marker = '+')
    #plt.vlines([ave, ave - np.sqrt(var), ave + np.sqrt(var)], 0, max(lis))
    print('Var', var)
    return var

def hill_fun(NTF, hc):
    lis = list(range(0, NTF + 100, 100))
    ret = {x: pow(x, hc)/(pow(940, hc) + pow(x, hc)) for x in lis}
    return ret

def simulator(nP, NTF, rates, steps, trajectories, jumps, species):
    
    d = {} # A container for the activation number!
    
    for nTF in range(0, NTF+1, jumps):
        # if nTF == 0:
        #     nTF = 1
        initial_state = {f'P{_}': (1 if _ == 0 else 0) for _ in range(nP+1)}
        initial_state.update({'TF': nTF, 'mRNA': 0})
        e = BiochemStem(initial_state, rates)
        
        # Forward

        for _ in range(nP):
            prop_fun = f'P{_}*TF*kf'
            delta = {f'P{_}': -1, f'P{_+1}': 1, 'TF': -1}
            e.add_reaction(f'P{_+1}f', prop_fun, delta)

        e.add_reaction('mRNAf', f'P{nP}*km', {'mRNA': 1})
        
        # Backward

        for _ in range(nP):
            prop_fun = f'P{_+1}*kb{_+1}'
            delta = {f'P{_}': 1, f'P{_+1}': -1, 'TF': 1}
            e.add_reaction(f'P{_}b', prop_fun, delta)

        e.add_reaction('mRNAb', f'mRNA*kd', {'mRNA': -1})
        
        e.assemble()
        # e.assembly
        
        w = BiochemSimul(e, steps, trajectories)
        w.meth_direct()
        
        _j = list(w.stem.assembly['species'].values())
        j = _j.index(species)
        #d.update({nTF: np.mean(w.state_tor[:, j, :])})
        d.update({nTF: {'Ave': time_ave(w, j, species), 'Var': variance(w,j)}})
        
        print(nTF)
        #nTFcurrent = w.state_tor[:, j, :]
        #nTFbound = nTF - nTFcurrent
        #d.update({nTF: np1.mean(nTFbound)})
    
    return d

In [ ]:
d = simulator(nP, NTF, rates, steps, trajectories, jumps, species)
print(d)

In [ ]:
_NTF = list(d.keys())
Ave = np.array([np.mean(d[i]['Ave']) for i in _NTF])
Std = np.sqrt(np.array([np.mean(d[i]['Var']) for i in _NTF]))
Ave[0] = 0
Std[0] = 0

h = hill_fun(NTF, 3.5)
#plt.plot(list(h.keys()), list(h.values()), color = 'orange')

plt.plot(_NTF, Ave, color = 'green', marker = '+')
plt.plot(_NTF, Ave + Std, color = 'pink', marker = '.')
plt.plot(_NTF, Ave - Std, color = 'pink', marker = '.')
#plt.hlines([0, 0.5, 1], 0, 2000, colors = 'lightgray', linestyles = 'dashed')
#plt.vlines(940, -0.25, 1.25, colors = 'gray', linestyles = 'dashed')
plt.savefig(f'{species}_{pre}.jpeg', dpi = 250, quality = 95)

In [ ]:
plt.plot(_NTF, Std / Ave, marker = '+')
plt.savefig(f'{species}_CV_{pre}.jpeg', dpi = 250, quality = 95)
plt.show()

plt.plot(_NTF, np.power(Std, 2) / Ave, marker = '+')
plt.hlines(0.1, 0, NTF)
plt.savefig(f'{species}_Fan_{pre}.jpeg', dpi = 250, quality = 95)
plt.show()

plt.plot(_NTF, Std, marker = '+')
plt.plot(_NTF, np.power(Std, 2), marker = '+')
plt.savefig(f'{species}_NTF_Std_{pre}.jpeg', dpi = 250, quality = 95)
plt.show()

plt.plot(Ave, Std)
plt.plot(Ave, np.power(Ave, 1/2))
plt.savefig(f'{species}_Ave_(Std_OR_sqrt(Ave))_{pre}.jpeg', dpi = 250, quality = 95)